In [53]:
import pandas as pd
import numpy as np

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [54]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [55]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [56]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [57]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [58]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,662,silver maple
2,18,pagoda dogwood
3,3917,littleleaf linden
4,12,American larch
...,...,...
128,362,sugar maple
129,170,Cornelian cherry
130,203,white oak
131,88,Kentucky yellowwood


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [59]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [60]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=boroname,spc_common,health').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,boroname,spc_common,health,count_tree_id
0,Bronx,Kentucky coffeetree,Poor,21
1,Bronx,golden raintree,Fair,76
2,Bronx,blackgum,Fair,9
3,Bronx,spruce,Fair,1
4,Bronx,false cypress,Poor,1
...,...,...,...,...
349,Bronx,common hackberry,Fair,132
350,Bronx,bald cypress,Good,65
351,Bronx,European hornbeam,Fair,89
352,Bronx,American beech,Good,21


# Solution:

## Step 1: Create Valid Selection List

The first step was to create a valid selection list for the dashboard. Not every tree is present in every borough, so I created a dictionary that lists each tree in each borough. I am using this selection list to create the dropdown menus.

In [61]:
#soql query to select all unique pairs of boroughs and tree species
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common' +\
        '&$group=boroname,spc_common'   ).replace(' ', '%20')

#run soql query into a pandas dataframe and drop null values
valid_selection = pd.read_json(soql_url).dropna()

#convert dataframe into dictionary
valid_key_pairs = valid_selection.groupby('boroname')['spc_common'].apply(list).to_dict()

valid_key_pairs

{'Bronx': ['American beech',
  'American elm',
  'American hophornbeam',
  'American hornbeam',
  'American larch',
  'American linden',
  'Amur cork tree',
  'Amur maackia',
  'Amur maple',
  'arborvitae',
  'ash',
  'Atlantic white cedar',
  'Atlas cedar',
  'bald cypress',
  'bigtooth aspen',
  'black cherry',
  'blackgum',
  'black locust',
  'black maple',
  'black oak',
  'black pine',
  'black walnut',
  'blue spruce',
  'boxelder',
  'bur oak',
  'Callery pear',
  'catalpa',
  'cherry',
  'Chinese chestnut',
  'Chinese elm',
  'Chinese fringetree',
  'Chinese tree lilac',
  'cockspur hawthorn',
  'common hackberry',
  'Cornelian cherry',
  'crab apple',
  'crepe myrtle',
  'crimson king maple',
  'cucumber magnolia',
  'dawn redwood',
  'Douglas-fir',
  'eastern cottonwood',
  'eastern hemlock',
  'eastern redbud',
  'eastern redcedar',
  'empress tree',
  'English oak',
  'European alder',
  'European beech',
  'European hornbeam',
  'false cypress',
  'flowering dogwood',
  '

## Step 2: Create functions to process api output

The next step was to create a couple functions to process the output from the soql api. There are a few missing values for trees that need to be removed and the sorting of the categories was in no particular order. The code below removes the missing values, reorders the categories, and calculates the percentages of trees by health status. The percentages allow us to standardize the comparisons of tree health across locales.

In [62]:
#function to process the first problem
def process_df(df):
    
    #drop na values
    output = df.dropna()
    
    #create a dataframe to sort tree health
    health_sort = pd.DataFrame({'health':['Poor','Fair','Good'],
                                'sort_order':[1,2,3]})
    
    #joins health sort dataframe with output dataframe
    joined = output.join(health_sort.set_index('health'), on ='health').sort_values(by=['sort_order'])
    
    #creates percentage column
    joined['percentage'] = round(joined['count_tree_id']/sum(joined['count_tree_id']),3)
    
    #creates tree count display column
    joined['count_tree_id_label'] = joined['count_tree_id'].astype('str') + ' Trees'
    
    return(joined)
    

In [63]:
#function to process the second problem
def process_df2(df):
    
    #drop na values
    df = df.dropna()
    
    #create dataframe of steward and tree count
    grouped = df.groupby(['steward'], as_index = False).sum()[['steward','count_tree_id']].rename(columns = {'count_tree_id':'tree_sum'})

    #join original dataframe with steward/tree count to calculate percentage
    joined = df.join(grouped.set_index('steward'), on ='steward')

    #divide tree count with sum of trees to calculate percentage of trees
    joined['percentage'] = (joined['count_tree_id']/joined['tree_sum']).round(3)

    #create tree count display column
    joined['count_tree_id_label'] = joined['count_tree_id'].astype('str') + ' Trees'
    
    #create a dataframe to sort tree health
    health_sort = pd.DataFrame({'health':['Poor','Fair','Good'],
                                'sort_order':[1,2,3]})

    #joins sort order onto tree health
    sorted_stuff = joined.join(health_sort.set_index('health'), on ='health').sort_values(by=['sort_order'])

    
    return(sorted_stuff)

## Step 3: Create functions to run soql queries

The next step was to create a couple functions that ran the soql queries and aggregated the results into a list of dataframes. This format makes it easy to call the data for the display output.

In [64]:
#runs all soql queries to find all borough and tree combos
def all_trees_in_boro(boro,tree_name):
    
    #soql query to find health of all trees in given borough
    boro_only_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,health,count(tree_id)' +\
        '&$where=boroname=\'{}\'' +\
        '&$group=boroname,health').format(boro).replace(' ', '%20')
    
    boro_only = process_df(pd.read_json(boro_only_soql))
  
    #soql query to find health of a tree species in all of NY
    tree_only_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,health,count(tree_id)' +\
        '&$where=spc_common=\'{}\'' +\
        '&$group=spc_common,health').format(tree_name).replace(' ', '%20')
    
    tree_only = process_df(pd.read_json(tree_only_soql))

    #soql query to find health of a tree species in a given borough
    boro_and_tree_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,count(tree_id)' +\
        '&$where=boroname=\'{}\' AND spc_common=\'{}\'' +\
        '&$group=boroname,spc_common,health').format(boro,tree_name).replace(' ', '%20')
    
    boro_and_tree = process_df(pd.read_json(boro_and_tree_soql))
    
    output = [boro_only,tree_only,boro_and_tree]

    return(output)

all_trees_in_boro('Bronx','American beech')

[  boroname  count_tree_id health  sort_order  percentage count_tree_id_label
 2    Bronx           3095   Poor           1       0.038          3095 Trees
 3    Bronx          10887   Fair           2       0.135         10887 Trees
 1    Bronx          66603   Good           3       0.826         66603 Trees,
        spc_common health  count_tree_id  sort_order  percentage  \
 1  American beech   Poor             21           1       0.077   
 2  American beech   Fair             38           2       0.139   
 0  American beech   Good            214           3       0.784   
 
   count_tree_id_label  
 1            21 Trees  
 2            38 Trees  
 0           214 Trees  ,
   boroname      spc_common health  count_tree_id  sort_order  percentage  \
 2    Bronx  American beech   Poor              3           1       0.097   
 0    Bronx  American beech   Fair              7           2       0.226   
 1    Bronx  American beech   Good             21           3       0.677   
 
  

In [65]:
#runs all soql queries to find health of trees by number of stewards
def all_stewards(boro,tree_name):
    
    #finds health of all trees in borough
    boro_only_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,steward,health,count(tree_id)' +\
        '&$where=boroname=\'{}\'' +\
        '&$group=boroname,steward,health').format(boro).replace(' ', '%20')

    boro_only = process_df2(pd.read_json(boro_only_soql))

    #finds health of all trees of a given species
    tree_only_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,steward,health,count(tree_id)' +\
        '&$where=spc_common=\'{}\'' +\
        '&$group=spc_common,steward,health').format(tree_name).replace(' ', '%20')

    tree_only = process_df2(pd.read_json(tree_only_soql))

    #finds health of trees in a borough of a given species
    boro_and_tree_soql = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,steward,health,count(tree_id)' +\
        '&$where=boroname=\'{}\' AND spc_common=\'{}\'' +\
        '&$group=boroname,spc_common,steward,health').format(boro,tree_name).replace(' ', '%20')

    boro_and_tree = process_df2(pd.read_json(boro_and_tree_soql))

    all_dfs = [boro_only, tree_only, boro_and_tree]

    return(all_dfs)

all_stewards('Bronx','American beech')

[   boroname  steward health  count_tree_id  tree_sum  percentage  \
 0     Bronx     None   Poor           2412     64851       0.037   
 8     Bronx     1or2   Poor            640     14808       0.043   
 9     Bronx  4orMore   Poor              2        71       0.028   
 10    Bronx     3or4   Poor             41       855       0.048   
 2     Bronx     None   Fair           8625     64851       0.133   
 6     Bronx  4orMore   Fair              7        71       0.099   
 7     Bronx     3or4   Fair            125       855       0.146   
 11    Bronx     1or2   Fair           2130     14808       0.144   
 3     Bronx     None   Good          53814     64851       0.830   
 4     Bronx  4orMore   Good             62        71       0.873   
 5     Bronx     3or4   Good            689       855       0.806   
 12    Bronx     1or2   Good          12038     14808       0.813   
 
    count_tree_id_label  sort_order  
 0           2412 Trees           1  
 8            640 Trees  

## Step 4: Create functions to format dataframes into dictionaries

The functions below format the dataframes from the previous functions into a dictionary that can be read by the app.

In [66]:
#creates display output dictionary for problem 1
def fetch_problem1(boro, tree_name):
    
    #creates all dataframes
    all_dfs = all_trees_in_boro(boro,tree_name)
    
    #list data elements into a list of dictionaries
    data = [{'x':list(all_dfs[2]["health"]), 
             'y':list(all_dfs[2]["percentage"]),
             'type':'bar',
             'name':tree_name + ' in ' + boro, 
             'text':list(all_dfs[2]["count_tree_id_label"])},
            
            {'x':list(all_dfs[0]["health"]), 
             'y':list(all_dfs[0]["percentage"]),
             'type':'bar',
             'name':'All trees in ' + boro, 
             'text':list(all_dfs[0]["count_tree_id_label"])},
            
            {'x':list(all_dfs[1]["health"]), 
             'y':list(all_dfs[1]["percentage"]),
             'type':'bar',
             'name':'All ' + tree_name + ' in NY', 
             'text':list(all_dfs[1]["count_tree_id_label"])}
            
           ]
    
    #specify graph titles
    title = {'title':'Health of ' + tree_name + ' in ' + boro,
             'xaxis':{'title':'Tree Health'},
             'yaxis':{'title':'Percentage'}}
    
    output_dict = {'data':data, 'layout':title}
    
    return(output_dict)


fetch_problem1('Bronx','American beech')

{'data': [{'x': ['Poor', 'Fair', 'Good'],
   'y': [0.097, 0.226, 0.677],
   'type': 'bar',
   'name': 'American beech in Bronx',
   'text': ['3 Trees', '7 Trees', '21 Trees']},
  {'x': ['Poor', 'Fair', 'Good'],
   'y': [0.038, 0.135, 0.826],
   'type': 'bar',
   'name': 'All trees in Bronx',
   'text': ['3095 Trees', '10887 Trees', '66603 Trees']},
  {'x': ['Poor', 'Fair', 'Good'],
   'y': [0.077, 0.139, 0.784],
   'type': 'bar',
   'name': 'All American beech in NY',
   'text': ['21 Trees', '38 Trees', '214 Trees']}],
 'layout': {'title': 'Health of American beech in Bronx',
  'xaxis': {'title': 'Tree Health'},
  'yaxis': {'title': 'Percentage'}}}

In [67]:
#creates display output dictionary for problem 2
def fetch_problem2(boro,tree_name):
    
    #fetches all dataframes
    stewards = all_stewards(boro,tree_name)[2]
    
    #split dataframes by steward count
    NoStewards  = stewards.loc[stewards['steward'] == 'None']
    OneOrTwo    = stewards.loc[stewards['steward'] == '1or2']
    ThreeOrFour = stewards.loc[stewards['steward'] == '3or4']
    FourOrMore  = stewards.loc[stewards['steward'] == '4orMore']

    #list data elements into a list of dictionaries
    data = [{'x':list(NoStewards["health"]), 
             'y':list(NoStewards["percentage"]),
             'type':'bar',
             'name':'No Stewards', 
             'text':list(NoStewards["count_tree_id_label"])},

            {'x':list(OneOrTwo["health"]), 
             'y':list(OneOrTwo["percentage"]),
             'type':'bar',
             'name':'1 or 2 Stewards', 
             'text':list(OneOrTwo["count_tree_id_label"])},

            {'x':list(ThreeOrFour["health"]), 
             'y':list(ThreeOrFour["percentage"]),
             'type':'bar',
             'name':'3 or 4 Stewards', 
             'text':list(ThreeOrFour["count_tree_id_label"])},

            {'x':list(FourOrMore["health"]), 
             'y':list(FourOrMore["percentage"]),
             'type':'bar',
             'name':'4 or More Stewards', 
             'text':list(FourOrMore["count_tree_id_label"])}
           ]

    #create titles and labels
    title = {'title':'Health of ' + tree_name + ' in ' + boro + ' by Steward Count',
             'xaxis':{'title':'Tree Health'},
             'yaxis':{'title':'Percentage'}}

    
    output_dict = {'data':data, 'layout':title}

    return(output_dict)

fetch_problem2('Queens','ash')

{'data': [{'x': ['Poor', 'Fair', 'Good'],
   'y': [0.032, 0.095, 0.873],
   'type': 'bar',
   'name': 'No Stewards',
   'text': ['38 Trees', '113 Trees', '1042 Trees']},
  {'x': ['Poor', 'Fair', 'Good'],
   'y': [0.034, 0.126, 0.84],
   'type': 'bar',
   'name': '1 or 2 Stewards',
   'text': ['4 Trees', '15 Trees', '100 Trees']},
  {'x': ['Poor', 'Fair', 'Good'],
   'y': [0.083, 0.167, 0.75],
   'type': 'bar',
   'name': '3 or 4 Stewards',
   'text': ['1 Trees', '2 Trees', '9 Trees']},
  {'x': [], 'y': [], 'type': 'bar', 'name': '4 or More Stewards', 'text': []}],
 'layout': {'title': 'Health of ash in Queens by Steward Count',
  'xaxis': {'title': 'Tree Health'},
  'yaxis': {'title': 'Percentage'}}}

## Step 5: Create the app

The last step is to create the app itself. Most of the code below is just formatting the webpage itself and managing the callbacks for each menu item selection.

In [68]:
#access css stylesheet for web layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#create app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#assign all valid key pairs
all_options = valid_key_pairs

#specify layout
app.layout = html.Div([
    
    #headers and formatting
    html.H1(children = 'Health of Trees in New York'),
    
    html.Div(children = 'Use this dashboard to study the health of various trees across New York.'),
    
    html.Hr(),
    
    #creates option to select a borough
    html.Div(children = 'Select a borough:'),
    
    dcc.RadioItems(
        id='boro-radio',
        options=[{'label': k, 'value': k} for k in all_options.keys()],
        value='Bronx'
    ),

    html.Hr(),
    
    #creates option to select a species of tree
    html.Div(children = 'Select a tree:'),
    
    dcc.Dropdown(id='tree-dropdown'),

    html.Hr(),
    
    html.Div(children = 'The graph below compares the health of a particular species of tree relative to all trees in a given borough and relative to that species of tree in New York.'),
    
    #creates graph for problem 1
    dcc.Graph(id = 'example-graph'),
    
    html.Hr(),
    
    html.Div(children = 'The graph below compares the health of a particular species of tree by number of stewards.'),
    
    #creates graph for problem 2
    dcc.Graph(id = 'graph2')
])

#specify input callback for first set of radio buttons
@app.callback(
    Output('tree-dropdown', 'options'),
    [Input('boro-radio', 'value')])
def set_trees_options(selected_boro):
    return [{'label': i, 'value': i} for i in all_options[selected_boro]]


#specify input callback for tree dropdown
@app.callback(
    Output('tree-dropdown', 'value'),
    [Input('tree-dropdown', 'options')])
def set_trees_value(available_options):
    return available_options[0]['value']

#specify input callback for problem 1 graph
@app.callback(
    Output('example-graph', 'figure'),
    [Input('boro-radio', 'value'),
     Input('tree-dropdown', 'value')])
def return_graph(selected_boro, selected_tree):
    return fetch_problem1(selected_boro, selected_tree)


#specify input callback for problem 2 graph
@app.callback(
    Output('graph2', 'figure'),
    [Input('boro-radio', 'value'),
     Input('tree-dropdown', 'value')])
def return_graph(selected_boro, selected_tree):
    return fetch_problem2(selected_boro, selected_tree)


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
